## Optimized `structure_pending` function using Cython
As an alternative to the `python-sc2` - `already_pending` function

In [1]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/GresvanAIE.xz"

In [2]:
# load cell magic things
%load_ext line_profiler
%load_ext Cython

In [3]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_data import Race
from sc2.unit import Unit
from sc2.units import Units

from tests.load_bot_from_pickle import get_map_specific_bot

In [4]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

Following is similar to `_abilities_count_and_build_progress` property found in `python-sc2`.
`already_pending` calls this and caches the result. No caching here as we just want to test performance.

In [6]:
from sc2.constants import CREATION_ABILITY_FIX
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from collections import Counter
from sc2.data import Race
from sc2.unit import Unit

def _abilities_count_and_build_progress(self):
    """Cache for the already_pending function, includes protoss units warping in,
    all units in production and all structures, and all morphs"""
    abilities_amount: Counter[AbilityId] = Counter()
    unit: Unit
    for unit in self.units + self.structures:
        for order in unit.orders:
            abilities_amount[order.ability.exact_id] += 1
        if not unit.is_ready and (self.race != Race.Terran or not unit.is_structure):
            # If an SCV is constructing a building, already_pending would count this structure twice
            # (once from the SCV order, and once from "not structure.is_ready")
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    # Hotfix for archons in morph state
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    abilities_amount[creation_ability] += 2
                else:
                    # Hotfix for rich geysirs
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    abilities_amount[creation_ability] += 1
            else:
                creation_ability: AbilityId = self.game_data.units[unit.type_id.value].creation_ability.exact_id
                abilities_amount[creation_ability] += 1

    return abilities_amount

In [7]:
%timeit _abilities_count_and_build_progress(bot)

18.3 μs ± 287 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Cythonize `abilities_count_structures` and `structure_pending`

In [8]:
%%cython

# cython: boundscheck=False, wraparound=False, cdivision=True
"""
Optimized Cython implementation for tracking ability counts and build progress.
Replaces the Python _abilities_count_and_build_progress method for maximum speed.
"""

from cython_extensions.ability_mapping import map_value

from sc2.data import Race
from sc2.ids.ability_id import AbilityId
from cython cimport boundscheck, wraparound
from libc.string cimport memset
from cpython.mem cimport PyMem_Malloc, PyMem_Free


cdef int STRUCT_ABILITIES[1620]  # raw C array

memset(STRUCT_ABILITIES, 0, 1620 * sizeof(int))
STRUCT_ABILITIES[AbilityId.BUILD_REACTOR_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.BUILD_TECHLAB_STARPORT.value] = 1
STRUCT_ABILITIES[AbilityId.TERRANBUILD_COMMANDCENTER.value] = 2
STRUCT_ABILITIES[AbilityId.ZERGBUILD_HATCHERY.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOLAIR_LAIR.value] = 2
STRUCT_ABILITIES[AbilityId.UPGRADETOHIVE_HIVE.value] = 1


cdef struct AbilityCount:
    int ability_id
    unsigned int count

cdef class AbilityBuffer:
    cdef AbilityCount* ptr
    cdef int size

    def __cinit__(self, int size):
        self.size = size
        self.ptr = <AbilityCount*> PyMem_Malloc(size * sizeof(AbilityCount))
        memset(self.ptr, 0, size * sizeof(AbilityCount))

    def __dealloc__(self):
        if self.ptr != NULL:
            PyMem_Free(self.ptr)
            self.ptr = NULL

    property mv:
        def __get__(self):
            # Return a Python memoryview
            return <AbilityCount[:self.size]> self.ptr


# Disable Python checks for speed
@boundscheck(False)
@wraparound(False)
cpdef AbilityCount[:] abilities_count_structures(object bot):
    """
    Build a C array indexed by ability_id that stores counts.
    Returns: memoryview of AbilityCount (size = 2200)
    """
    cdef:
        unsigned int MIN_ABILITIES = 0
        unsigned int MAX_ABILITIES = 2200
        AbilityBuffer buf = AbilityBuffer(MAX_ABILITIES)
        AbilityCount* arr = buf.ptr
        object unit, order, orders
        int ability_id



        object structures = bot.structures
        object workers = bot.workers
        bint is_protoss = bot.race == Race.Protoss

        bint is_zerg = bot.race == Race.Zerg

        unsigned int len_structures = len(structures)
        unsigned int len_workers = len(workers)
        unsigned int len_orders, i, j, struct_ability_int, unit_id_int

        double completed_build_progress = 1.0


    #FUTURE exclude this for ares?
    for i in range(len_workers):
        unit = workers[i]
        orders = unit.orders
        len_orders = len(orders)
        for j in range(len_orders):
            order = orders[j]
            ability_id = <int> order.ability._proto.ability_id
            if 0 <= ability_id < MAX_ABILITIES:
                arr[ability_id].count += 1


    # Structures → build progress < 1.0 → increment creation ability
    if is_protoss:
        for i in range(len_structures):
            unit = structures[i]
            if <double> unit._proto.build_progress < completed_build_progress:
                unit_id_int = <int> unit._proto.unit_type
                ability_id = <int> map_value(unit_id_int)
                if ability_id!=-1:
                    arr[ability_id].count += 1

    elif is_zerg:
        #Terran and Zerg
        for i in range(len_structures):
            unit = structures[i]
            unit_id_int = <int> unit._proto.unit_type
            ability_id = <int> map_value(unit_id_int)
            if <double> unit._proto.build_progress < 1.0:


                if ability_id!=-1:
                    arr[ability_id].count += 1
            elif STRUCT_ABILITIES[ability_id]==2:  #identify Lair, Hive and Command Center
                # Lair and Hive
                orders = unit.orders
                len_orders = len(orders)
                for j in range(len_orders):
                    order = orders[j]
                    ability_id = <int> order.ability._proto.ability_id
                    if STRUCT_ABILITIES[ability_id]:
                        arr[ability_id].count += 1

    else:
        for i in range(len_structures):
            unit = structures[i]
            unit_id_int = <int> unit._proto.unit_type
            ability_id = <int> map_value(unit_id_int)
            if <double> unit._proto.build_progress < 1.0:


                if STRUCT_ABILITIES[ability_id]:
                    arr[ability_id].count += 1

            elif STRUCT_ABILITIES[ability_id]==2:  #identify Commandcenter, but in the same way as others to save time
             # Command Center for OC and PF
                orders = unit.orders
                len_orders = len(orders)
                for j in range(len_orders):
                    order = orders[j]
                    ability_id = <int> order.ability._proto.ability_id
                    if STRUCT_ABILITIES[ability_id]:
                        arr[ability_id].count += 1

    # Return as Python-usable memoryview
    return buf.mv


def cache_per_game_loop(func):
    cache_name = f"_{func.__name__}_cache"
    loop_name = f"_{func.__name__}_loop"
    def wrapper(bot, *args, **kwargs):
        current_loop = bot.state.game_loop
        if getattr(bot, loop_name, None) == current_loop:
            return getattr(bot, cache_name)
        result = func(bot, *args, **kwargs)
        setattr(bot, cache_name, result)
        setattr(bot, loop_name, current_loop)
        return result
    return wrapper

@cache_per_game_loop
def cy_abilities_count_structures(bot):
    return abilities_count_structures(bot)


@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        AbilityCount[:] counts_and_progress
        int target = <int> structure_type.value
        AbilityCount item
        Py_ssize_t arr_len
        int target_created_ability



    # Use optimized Cython function to get ability counts

    counts_and_progress = cy_abilities_count_structures(bot) #returns a c array memoryview

    arr_len = counts_and_progress.shape[0]
    target_created_ability = <int> map_value(target)

    if 0 <= target_created_ability < arr_len:
        item = counts_and_progress[target_created_ability]
        num_pending += item.count
    return num_pending

### Test caching abilities_count_structures

In [9]:
bot.state.game_loop += 1

In [10]:
def test_cache_burny(bot):
    bot.state.game_loop += 1
    bot._abilities_count_and_build_progress

def test_cache_cython(bot):
    bot.state.game_loop += 1
    cy_abilities_count_structures(bot)

In [11]:
%timeit test_cache_burny(bot)

19.9 μs ± 284 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%timeit test_cache_cython(bot)

4.59 μs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### About 4x faster to cache then original function

### Test performance of structure_pending in realistic conditions

Simulate user calling structure_pending 8 times per game loop. First time is cached and subsequent times are not.

In [13]:
def test_pending_burny(bot, times_called):
    bot.state.game_loop += 1
    for _ in range(times_called):
        bot.already_pending(UnitTypeId.BARRACKS)

def test_pending_cython(bot, times_called):
    bot.state.game_loop += 1
    for _ in range(times_called):
        cy_structure_pending(bot, UnitTypeId.BARRACKS)


In [25]:
%timeit test_pending_burny(bot, 1)

22.6 μs ± 363 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%timeit test_pending_cython(bot, 1)

5.18 μs ± 51.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


#### About 4 times faster to call then original function

#### `cy_structure_pending` roughly 4x faster then original `already_pending` function


In [43]:
%timeit -n 1 -r 1 bot._abilities_count_and_build_progress

100 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
%timeit cy_structure_pending(bot, UnitTypeId.BARRACKS)

342 ns ± 7.01 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
%timeit bot.already_pending(UnitTypeId.BARRACKS)

1.77 μs ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [9]:
%timeit cy_abilities_count_structures(bot)

116 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
%timeit _abilities_count_and_build_progress(bot)

19 μs ± 330 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [258]:
%timeit bot._abilities_count_and_build_progress

177 ns ± 0.525 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [307]:
bot.state.game_loop += 1

In [308]:
%timeit cy_structure_pending(bot, UnitTypeId.BARRACKS)

4.16 μs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [309]:
%timeit bot.already_pending(UnitTypeId.BARRACKS)

1.73 μs ± 7.53 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [151]:
%timeit cy_structure_pending(bot, UnitTypeId.BARRACKS)

970 ns ± 74.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [152]:
%timeit bot.already_pending(UnitTypeId.BARRACKS)

1.82 μs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [77]:
worker=bot.workers.first

In [25]:
%timeit worker.orders[0].ability.exact_id

525 ns ± 47.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [26]:
%timeit worker.orders[0].ability._proto.ability_id

127 ns ± 1.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [34]:
worker.orders[0].ability.exact_id.value

295

In [35]:
worker.orders[0].ability._proto.ability_id

295

In [79]:
%timeit worker.type_id.value

425 ns ± 7.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [81]:
%timeit worker._proto.unit_type

90.9 ns ± 0.235 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
structure = bot.structures.first

In [6]:
%timeit structure.build_progress

143 ns ± 5.52 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
%timeit structure._proto.build_progress

117 ns ± 2.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
